In [1]:
# import pandas as pd

# # Read the CSV
# df = pd.read_csv(r"D:\Aviation\IMSR\final_df_resource_summary_AND_gacc_activity_AND_national_activity_2015_2021_WITH_water_lines_cooridors.csv")

# # Print column names
# print(df.columns.tolist())
import pandas as pd

# Read the CSV
df = pd.read_csv(r"D:\Aviation\IMSR\final_df_resource_summary_AND_gacc_activity_AND_national_activity_2015_2021_WITH_water_lines_cooridors.csv")

# Print column names
#print(df.columns.tolist())

df = df[
    (df['RPL_THEMES'] != -999) &
    (df['EPL_MUNIT'] != -999)
]

mapping = {'LIGHT': 0, 'MODERATE': 1, 'HEAVY': 2}
df['initial_attack_activity'] = df['initial_attack_activity'].map(mapping)



matched = df

import numpy as np
# Define variable sets
hazard_vars = [
    'vpd_Normal', 'bi_Normal', 'fm100_Normal',
    'srad_Normal', 'sph_Normal', 'Slope', 'TRI', 'TPI', 'Elevation',
    'erc', 'vs', 'Aridity_index', 'SDI'
]

values_vars = [
    'Popo_1km', 'GHM', 'RPL_THEMES', 'EPL_MUNIT', 'dist_to_water_m', 'dist_to_transline_m'
]

capacity_vars = [
    'NPL', 'GACC_PL', 'new_large_fires', 'type_2_teams',
    'type_1_teams', 'nimos', 'initial_attack_activity', 'engines', 'helicopters', 'crews'
]

predictors = hazard_vars + values_vars + capacity_vars

# # Define placeholder values and problematic columns
# placeholder_vals = [-999, -999.0, -3.402823e+38]
# problem_columns = ['SDI', 'GHM', 'RPL_THEMES', 'EPL_MUNIT']

# # Print length before cleaning
# print(f"Original matched length: {len(matched)}")

# # Replace known placeholder values with NaN
# for col in problem_columns:
#     matched[col] = matched[col].replace(placeholder_vals, np.nan)

print(len(matched))

# Drop rows where SDI or GHM are < 0
#matched = matched[(matched['SDI'] > 0) & (matched['GHM'] > 0)]

matched['SDI'] = matched['SDI'].where(matched['SDI'] >= 0, 0)
matched['GHM'] = matched['GHM'].where(matched['GHM'] >= 0, 0)

# Drop rows with missing predictor or target values
matched_clean = matched.dropna(subset=predictors + ['event_type'])

# Print length after cleaning
print(f"Cleaned matched_clean length: {len(matched_clean)}")

# Print unique event_type values and their counts
event_counts = matched_clean['event_type'].value_counts()
print("Event type counts:")
print(event_counts)

# Print number of unique event types
print(f"\nNumber of unique event types: {matched_clean['event_type'].nunique()}")



C:\Users\magst\AppData\Local\Temp\ipykernel_5560\3623525150.py:11: DtypeWarning: Columns (14,15,16,17,311) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"D:\Aviation\IMSR\final_df_resource_summary_AND_gacc_activity_AND_national_activity_2015_2021_WITH_water_lines_cooridors.csv")


455960
Cleaned matched_clean length: 445107
Event type counts:
event_type
0    435433
1      9674
Name: count, dtype: int64

Number of unique event types: 2


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# # Define predictor sets
# hazard_vars = [
#     'vpd_Normal', 'bi_Normal', 'fm100_Normal', 'srad_Normal', 'sph_Normal',
#     'Slope', 'TRI', 'TPI', 'Elevation', 'erc', 'vs', 'Aridity_index', 'SDI'
# ]
# values_vars = ['Popo_1km', 'GHM', 'RPL_THEMES', 'EPL_MUNIT']
# capacity_vars = [
#     'NPL', 'GACC_PL', 'new_large_fires', 'type_2_teams', 'type_1_teams',
#     'nimos', 'initial_attack_activity', 'engines', 'helicopters', 'crews'
# ]
# predictors = hazard_vars + values_vars + capacity_vars

# Descriptive statistics table
desc = matched_clean[predictors].describe().T[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]
desc.to_csv('descriptive_stats_table.csv')
print(desc)

# Correlation matrix plot
corr = matched_clean[predictors].corr()
fig, ax = plt.subplots(figsize=(12, 10))
cax = ax.imshow(corr, interpolation='nearest')
ax.set_xticks(range(len(predictors)))
ax.set_yticks(range(len(predictors)))
ax.set_xticklabels(predictors, rotation=90)
ax.set_yticklabels(predictors)
fig.colorbar(cax)
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.show()


In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# corr = matched_clean[predictors].corr()
# plt.figure(figsize=(12,10))
# sns.heatmap(corr, annot=False, cmap='coolwarm', center=0)

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

X_vif = sm.add_constant(matched_clean[predictors])
vif_data = pd.DataFrame()
vif_data['feature'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i)
                   for i in range(X_vif.shape[1])]

print(vif_data)


In [ ]:
matched_clean['sph_Normal'].describe()


In [ ]:
import numpy as np
# Define variable sets

matched_clean['sph_Normal_1000'] = matched_clean['sph_Normal'] * 1000


hazard_vars = [
    'vpd_Normal', 'bi_Normal',
    'srad_Normal', 'sph_Normal_1000', 'TPI', 'Elevation',
    'erc', 'vs', 'Aridity_index', 'SDI'
]

values_vars = [
    'Popo_1km', 'GHM', 'RPL_THEMES', 'EPL_MUNIT', 'dist_to_water_m', 'dist_to_transline_m'
]

capacity_vars = [
    'NPL', 'GACC_PL', 'new_large_fires', 'type_2_teams',
    'type_1_teams', 'nimos', 'initial_attack_activity', 'engines', 'helicopters', 'crews'
]

predictors = hazard_vars + values_vars + capacity_vars


In [ ]:
# import pandas as pd
# import statsmodels.api as sm
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score, f1_score, classification_report
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler


# # Prepare data
# matched_clean = matched_clean.copy()  # ensure fresh copy
# X = matched_clean[predictors]
# y = matched_clean['event_type']
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

# # Scale features using training data only
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


# # 1) Statsmodels Logit
# X_train_sm = sm.add_constant(X_train)
# logit_train = sm.Logit(y_train, X_train_sm).fit(disp=False)
# print(logit_train.summary())

# X_test_sm = sm.add_constant(X_test)
# y_proba = logit_train.predict(X_test_sm)
# print("AUC (Logit):", roc_auc_score(y_test, y_proba))

# best_t, best_f = 0, 0
# for t in [i/100 for i in range(1,100)]:
#     f = f1_score(y_test, (y_proba > t).astype(int))
#     if f > best_f:
#         best_f, best_t = f, t
# print(f"Best threshold: {best_t}, F1: {best_f}")
# print(classification_report(y_test, (y_proba > best_t).astype(int)))

# # 2) Class-weighted Logistic Regression
# lr = LogisticRegression(class_weight='balanced', max_iter=1000)
# lr.fit(X_train, y_train)
# y_proba_w = lr.predict_proba(X_test)[:,1]
# print("AUC (class_weight LR):", roc_auc_score(y_test, y_proba_w))

# best_t_w, best_f_w = 0, 0
# for t in [i/100 for i in range(1,100)]:
#     f = f1_score(y_test, (y_proba_w > t).astype(int))
#     if f > best_f_w:
#         best_f_w, best_t_w = f, t
# print(f"Best threshold (weighted): {best_t_w}, F1: {best_f_w}")
# print(classification_report(y_test, (y_proba_w > best_t_w).astype(int)))

# # 3) Manual Undersampling
# train_df = pd.concat([X_train, y_train], axis=1)
# maj = train_df[train_df['event_type']==0]
# minn = train_df[train_df['event_type']==1]
# maj_down = maj.sample(n=len(minn), random_state=42)
# balanced = pd.concat([maj_down, minn])
# X_bal, y_bal = balanced[predictors], balanced['event_type']
# lr2 = LogisticRegression(max_iter=1000)
# lr2.fit(X_bal, y_bal)
# y_proba_s = lr2.predict_proba(X_test)[:,1]
# print("AUC (undersampled LR):", roc_auc_score(y_test, y_proba_s))

# best_t_s, best_f_s = 0, 0
# for t in [i/100 for i in range(1,100)]:
#     f = f1_score(y_test, (y_proba_s > t).astype(int))
#     if f > best_f_s:
#         best_f_s, best_t_s = f, t
# print(f"Best threshold (undersampled): {best_t_s}, F1: {best_f_s}")
# print(classification_report(y_test, (y_proba_s > best_t_s).astype(int)))


In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler

# Prepare data
matched_clean = matched_clean.copy()
X = matched_clean[predictors]
y = matched_clean['event_type']
X_train_df, X_test_df, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_df), columns=X.columns, index=X_train_df.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_df), columns=X.columns, index=X_test_df.index)

# 1) Statsmodels Logit (with named features)
X_train_sm = sm.add_constant(X_train_scaled)
logit_train = sm.Logit(y_train, X_train_sm).fit(disp=False)
print(logit_train.summary())

X_test_sm = sm.add_constant(X_test_scaled)
y_proba = logit_train.predict(X_test_sm)
print("AUC (Logit):", roc_auc_score(y_test, y_proba))

best_t, best_f = 0, 0
for t in [i / 100 for i in range(1, 100)]:
    f = f1_score(y_test, (y_proba > t).astype(int))
    if f > best_f:
        best_f, best_t = f, t
print(f"Best threshold: {best_t}, F1: {best_f}")
print(classification_report(y_test, (y_proba > best_t).astype(int)))

# 2) Class-weighted Logistic Regression
lr = LogisticRegression(class_weight='balanced', max_iter=1000)
lr.fit(X_train_scaled, y_train)
y_proba_w = lr.predict_proba(X_test_scaled)[:, 1]
print("AUC (class_weight LR):", roc_auc_score(y_test, y_proba_w))

best_t_w, best_f_w = 0, 0
for t in [i / 100 for i in range(1, 100)]:
    f = f1_score(y_test, (y_proba_w > t).astype(int))
    if f > best_f_w:
        best_f_w, best_t_w = f, t
print(f"Best threshold (weighted): {best_t_w}, F1: {best_f_w}")
print(classification_report(y_test, (y_proba_w > best_t_w).astype(int)))

# 3) Manual Undersampling (using scaled data with column names preserved)
train_df = pd.concat([X_train_scaled, y_train], axis=1)
maj = train_df[train_df['event_type'] == 0]
minn = train_df[train_df['event_type'] == 1]
maj_down = maj.sample(n=len(minn), random_state=42)
balanced = pd.concat([maj_down, minn])

X_bal = balanced.drop(columns='event_type')
y_bal = balanced['event_type']

lr2 = LogisticRegression(max_iter=1000)
lr2.fit(X_bal, y_bal)
y_proba_s = lr2.predict_proba(X_test_scaled)[:, 1]
print("AUC (undersampled LR):", roc_auc_score(y_test, y_proba_s))

best_t_s, best_f_s = 0, 0
for t in [i / 100 for i in range(1, 100)]:
    f = f1_score(y_test, (y_proba_s > t).astype(int))
    if f > best_f_s:
        best_f_s, best_t_s = f, t
print(f"Best threshold (undersampled): {best_t_s}, F1: {best_f_s}")
print(classification_report(y_test, (y_proba_s > best_t_s).astype(int)))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler

# Prepare data
matched_clean = matched_clean.copy()
X = matched_clean[predictors]
y = matched_clean['event_type']
X_train_df, X_test_df, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_df), columns=X.columns, index=X_train_df.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_df), columns=X.columns, index=X_test_df.index)

# Class-weighted Logistic Regression
lr = LogisticRegression(class_weight='balanced', max_iter=1000)
lr.fit(X_train_scaled, y_train)
y_proba = lr.predict_proba(X_test_scaled)[:, 1]

# AUC
print("AUC (class_weight LR):", roc_auc_score(y_test, y_proba))

# Find best threshold by F1
best_t, best_f = 0, 0
for t in [i / 100 for i in range(1, 100)]:
    f = f1_score(y_test, (y_proba > t).astype(int))
    if f > best_f:
        best_f, best_t = f, t
print(f"Best threshold (weighted): {best_t}, F1: {best_f}")

# Classification report at best threshold
print(classification_report(y_test, (y_proba > best_t).astype(int)))


In [ ]:
# FULL 

In [20]:
import numpy as np
# Define variable sets

matched_clean['sph_Normal_1000'] = matched_clean['sph_Normal'] * 1000


hazard_vars = [
    'vpd_Normal', 'bi_Normal',
    'srad_Normal', 'sph_Normal_1000', 'TPI', 'Elevation',
    'erc', 'vs', 'Aridity_index', 'SDI'
]

values_vars = [
    'Popo_1km', 'GHM', 'RPL_THEMES', 'EPL_MUNIT', 'dist_to_water_m', 'dist_to_transline_m'
]

capacity_vars = [
    'NPL', 'GACC_PL', 'new_large_fires', 'type_2_teams',
    'type_1_teams', 'nimos', 'initial_attack_activity', 'engines', 'helicopters', 'crews'
]

predictors = hazard_vars + values_vars + capacity_vars


In [12]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import roc_auc_score, f1_score
# import tensorflow as tf
# import keras_tuner as kt
# from tensorflow.keras import layers, models, callbacks
# from sklearn.utils.class_weight import compute_class_weight
# import os, shutil

# # assume matched_clean (DataFrame) and predictors (list of column names) are defined
# X = matched_clean[predictors]
# y = matched_clean['event_type']

# class F1Metric(tf.keras.metrics.Metric):
#     def __init__(self, name='f1', threshold=0.5, **kwargs):
#         super().__init__(name=name, **kwargs)
#         self.threshold = threshold
#         self.tp = self.add_weight(name='tp', initializer='zeros')
#         self.fp = self.add_weight(name='fp', initializer='zeros')
#         self.fn = self.add_weight(name='fn', initializer='zeros')
#     def update_state(self, y_true, y_pred, sample_weight=None):
#         y_pred = tf.cast(y_pred > self.threshold, tf.float32)
#         y_true = tf.cast(y_true, tf.float32)
#         self.tp.assign_add(tf.reduce_sum(y_true * y_pred))
#         self.fp.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
#         self.fn.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))
#     def result(self):
#         precision = self.tp / (self.tp + self.fp + 1e-8)
#         recall    = self.tp / (self.tp + self.fn + 1e-8)
#         return 2 * precision * recall / (precision + recall + 1e-8)
#     def reset_states(self):
#         for v in self.variables:
#             v.assign(0)

# def build_model(hp):
#     hp_units = hp.Int('units', min_value=16, max_value=256, step=16)
#     hp_lr    = hp.Float('lr', 1e-4, 1e-2, sampling='log')
#     model = models.Sequential([
#         layers.Input(shape=(X_train_scaled.shape[1],)),
#         layers.Dense(hp_units, activation='relu'),
#         layers.Dense(1, activation='sigmoid')
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr),
#         loss='binary_crossentropy',
#         metrics=[tf.keras.metrics.AUC(name='auc'), F1Metric()]
#     )
#     return model

# sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
# lr_auc_scores, lr_f1_scores = [], []
# nn_auc_scores, nn_f1_scores = [], []

# for i, (train_idx, test_idx) in enumerate(sss.split(X, y)):
#     train_df = matched_clean.iloc[train_idx]
#     test_df  = matched_clean.iloc[test_idx]
#     X_train, y_train = train_df[predictors], train_df['event_type']
#     X_test,  y_test  = test_df[predictors],  test_df['event_type']

#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled  = scaler.transform(X_test)

#     # Logistic Regression
#     lr = LogisticRegression(class_weight='balanced', max_iter=1000)
#     lr.fit(X_train_scaled, y_train)
#     y_proba_lr = lr.predict_proba(X_test_scaled)[:,1]
#     auc_lr = roc_auc_score(y_test, y_proba_lr)
#     best_f1_lr, best_t_lr = 0, 0
#     for t in np.linspace(0.01, 0.99, 99):
#         f = f1_score(y_test, (y_proba_lr > t).astype(int))
#         if f > best_f1_lr:
#             best_f1_lr, best_t_lr = f, t
#     lr_auc_scores.append(auc_lr)
#     lr_f1_scores.append(best_f1_lr)

#     # Neural Network
#     classes = np.unique(y_train)
#     weights = compute_class_weight('balanced', classes=classes, y=y_train)
#     class_weights = dict(zip(classes, weights))
#     tuner_dir = f"keras_tuner_dir/run_{i}"
#     if os.path.exists(tuner_dir): shutil.rmtree(tuner_dir)
#     tuner = kt.RandomSearch(
#         build_model, objective='val_auc', max_trials=10,
#         directory=tuner_dir, project_name='nn_run'
#     )
#     tuner.search(
#         X_train_scaled, y_train,
#         epochs=5, batch_size=512, validation_split=0.2,
#         callbacks=[callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
#         class_weight=class_weights, verbose=0
#     )
#     best_model = tuner.get_best_models(1)[0]
#     y_proba_nn = best_model.predict(X_test_scaled).flatten()
#     auc_nn = roc_auc_score(y_test, y_proba_nn)
#     best_f1_nn, best_t_nn = 0, 0
#     for t in np.linspace(0.01, 0.99, 99):
#         f = f1_score(y_test, (y_proba_nn > t).astype(int))
#         if f > best_f1_nn:
#             best_f1_nn, best_t_nn = f, t
#     nn_auc_scores.append(auc_nn)
#     nn_f1_scores.append(best_f1_nn)

# print(f"LR AUC: {np.mean(lr_auc_scores):.3f} ± {np.std(lr_auc_scores):.3f}")
# print(f"LR F1 : {np.mean(lr_f1_scores):.3f} ± {np.std(lr_f1_scores):.3f}")
# print(f"NN AUC: {np.mean(nn_auc_scores):.3f} ± {np.std(nn_auc_scores):.3f}")
# print(f"NN F1 : {np.mean(nn_f1_scores):.3f} ± {np.std(nn_f1_scores):.3f}")


Traceback (most recent call last):
  File "C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras_tuner\src\engine\hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)
  File "C:\User

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras_tuner\src\engine\hypermodel.py", 

In [22]:
# Logistic regression

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler

# Prepare data
matched_clean = matched_clean.copy()
X = matched_clean[predictors]
y = matched_clean['event_type']
X_train_df, X_test_df, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)


y_train = y_train.reset_index(drop=True)


# Scale features
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_df), columns=X.columns, index=X_train_df.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_df), columns=X.columns, index=X_test_df.index)

# Class-weighted Logistic Regression
lr = LogisticRegression(class_weight='balanced', max_iter=1000)
lr.fit(X_train_scaled, y_train)
y_proba = lr.predict_proba(X_test_scaled)[:, 1]

# AUC
print("AUC (class_weight LR):", roc_auc_score(y_test, y_proba))

# Find best threshold by F1
best_t, best_f = 0, 0
for t in [i / 100 for i in range(1, 100)]:
    f = f1_score(y_test, (y_proba > t).astype(int))
    if f > best_f:
        best_f, best_t = f, t
print(f"Best threshold (weighted): {best_t}, F1: {best_f}")

# Classification report at best threshold
print(classification_report(y_test, (y_proba > best_t).astype(int)))




# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import roc_auc_score, f1_score, classification_report

# # Initialize storage
# auc_scores = []
# f1_scores = []
# thresholds = []
# reports = []

# # Stratified splitting setup
# sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

# for i, (train_idx, test_idx) in enumerate(sss.split(matched_clean, matched_clean['event_type'])):
#     train_df = matched_clean.iloc[train_idx]
#     test_df = matched_clean.iloc[test_idx]

#     X_train = train_df[predictors]
#     y_train = train_df['event_type'].reset_index(drop=True)

#     X_test = test_df[predictors]
#     y_test = test_df['event_type'].reset_index(drop=True)

#     # Scale features
#     scaler = StandardScaler()
#     X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
#     X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

#     # Train class-weighted logistic regression
#     lr = LogisticRegression(class_weight='balanced', max_iter=1000)
#     lr.fit(X_train_scaled, y_train)
#     y_proba = lr.predict_proba(X_test_scaled)[:, 1]

#     # Compute AUC
#     auc = roc_auc_score(y_test, y_proba)
#     auc_scores.append(auc)

#     # Find best threshold for F1
#     best_f1, best_t = 0, 0
#     for t in np.linspace(0.01, 0.99, 99):
#         preds = (y_proba > t).astype(int)
#         f = f1_score(y_test, preds)
#         if f > best_f1:
#             best_f1 = f
#             best_t = t

#     f1_scores.append(best_f1)
#     thresholds.append(best_t)

#     # Save classification report
#     report = classification_report(y_test, (y_proba > best_t).astype(int), output_dict=True)
#     reports.append(report)

# # Summary
# print(f"\nLogistic Regression over 10 runs:")
# print(f"AUC: {np.mean(auc_scores):.3f} ± {np.std(auc_scores):.3f}")
# print(f"F1:  {np.mean(f1_scores):.3f} ± {np.std(f1_scores):.3f}")
# print(f"Best thresholds (mean): {np.mean(thresholds):.2f}")


AUC (class_weight LR): 0.79576778949017
Best threshold (weighted): 0.75, F1: 0.14549116826774094
              precision    recall  f1-score   support

           0       0.98      0.93      0.96    130631
           1       0.09      0.32      0.15      2902

    accuracy                           0.92    133533
   macro avg       0.54      0.63      0.55    133533
weighted avg       0.96      0.92      0.94    133533



In [ ]:
# # Logistic regression

# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score, f1_score, classification_report
# from sklearn.preprocessing import StandardScaler

# # Prepare data
# matched_clean = matched_clean.copy()
# X = matched_clean[predictors]
# y = matched_clean['event_type']
# X_train_df, X_test_df, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)


# y_train = y_train.reset_index(drop=True)


# # Scale features
# scaler = StandardScaler()
# X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_df), columns=X.columns, index=X_train_df.index)
# X_test_scaled = pd.DataFrame(scaler.transform(X_test_df), columns=X.columns, index=X_test_df.index)

# # Class-weighted Logistic Regression
# lr = LogisticRegression(class_weight='balanced', max_iter=1000)
# lr.fit(X_train_scaled, y_train)
# y_proba = lr.predict_proba(X_test_scaled)[:, 1]

# # AUC
# print("AUC (class_weight LR):", roc_auc_score(y_test, y_proba))

# # Find best threshold by F1
# best_t, best_f = 0, 0
# for t in [i / 100 for i in range(1, 100)]:
#     f = f1_score(y_test, (y_proba > t).astype(int))
#     if f > best_f:
#         best_f, best_t = f, t
# print(f"Best threshold (weighted): {best_t}, F1: {best_f}")

# # Classification report at best threshold
# print(classification_report(y_test, (y_proba > best_t).astype(int)))


In [23]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import layers, models, callbacks
from sklearn.metrics import roc_auc_score, f1_score, classification_report
import numpy as np



from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))


# --- Custom F1 metric ---------------------------------------------------------
class F1Metric(tf.keras.metrics.Metric):
    def __init__(self, name='f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fp = self.add_weight(name='fp', initializer='zeros')
        self.fn = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        y_true = tf.cast(y_true, tf.float32)
        self.tp.assign_add(tf.reduce_sum(y_true * y_pred))
        self.fp.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
        self.fn.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

    def result(self):
        precision = self.tp / (self.tp + self.fp + 1e-8)
        recall   = self.tp / (self.tp + self.fn + 1e-8)
        return 2 * precision * recall / (precision + recall + 1e-8)

    def reset_states(self):
        for v in self.variables:
            v.assign(0)

# --- Model builder -----------------------------------------------------------
def build_model(hp):
    hp_units = hp.Int('units', min_value=16, max_value=256, step=16)
    hp_lr    = hp.Float('lr', 1e-4, 1e-2, sampling='log')
    model = models.Sequential([
        layers.Input(shape=(X_train_scaled.shape[1],)),
        layers.Dense(hp_units, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc'), F1Metric()]
    )
    return model

# --- Hyperparameter tuning ----------------------------------------------------
tuner = kt.RandomSearch(
    build_model,
    objective='val_auc',
    max_trials=20,
    directory='keras_tuner_dir',
    project_name='one_hidden_layer_5'
)

early_stop = callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=0)

tuner.search(
    X_train_scaled, y_train,
    epochs=3,
    batch_size=512,
    validation_split=0.2,
    callbacks=[early_stop],
    class_weight=class_weights,    
    verbose=1
)

# --- Evaluation ---------------------------------------------------------------
best_model = tuner.get_best_models(1)[0]

# Print model architecture
best_model.summary()
best_hp = tuner.get_best_hyperparameters(1)[0]

# Print all tuned hyperparameters
print("Best hyperparameters:")
for param in best_hp.values:
    print(f"{param}: {best_hp.get(param)}")

    

y_proba_nn = best_model.predict(X_test_scaled).flatten()

print("AUC (NN):", roc_auc_score(y_test, y_proba_nn))

best_t_nn, best_f_nn = 0, 0
for t in np.linspace(0.01, 0.99, 99):
    f = f1_score(y_test, (y_proba_nn > t).astype(int))
    if f > best_f_nn:
        best_f_nn, best_t_nn = f, t

print(f"Best threshold (NN): {best_t_nn}, F1: {best_f_nn}")
print(classification_report(y_test, (y_proba_nn > best_t_nn).astype(int)))



# import tensorflow as tf
# import keras_tuner as kt
# from tensorflow.keras import layers, models, callbacks
# from sklearn.metrics import roc_auc_score, f1_score, classification_report
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.utils.class_weight import compute_class_weight
# import numpy as np
# import os
# import shutil

# # --- Custom F1 metric ---------------------------------------------------------
# class F1Metric(tf.keras.metrics.Metric):
#     def __init__(self, name='f1', threshold=0.5, **kwargs):
#         super().__init__(name=name, **kwargs)
#         self.threshold = threshold
#         self.tp = self.add_weight(name='tp', initializer='zeros')
#         self.fp = self.add_weight(name='fp', initializer='zeros')
#         self.fn = self.add_weight(name='fn', initializer='zeros')

#     def update_state(self, y_true, y_pred, sample_weight=None):
#         y_pred = tf.cast(y_pred > self.threshold, tf.float32)
#         y_true = tf.cast(y_true, tf.float32)
#         self.tp.assign_add(tf.reduce_sum(y_true * y_pred))
#         self.fp.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
#         self.fn.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

#     def result(self):
#         precision = self.tp / (self.tp + self.fp + 1e-8)
#         recall   = self.tp / (self.tp + self.fn + 1e-8)
#         return 2 * precision * recall / (precision + recall + 1e-8)

#     def reset_states(self):
#         for v in self.variables:
#             v.assign(0)

# # --- Model builder -----------------------------------------------------------
# def build_model(hp):
#     hp_units = hp.Int('units', min_value=16, max_value=256, step=16)
#     hp_lr    = hp.Float('lr', 1e-4, 1e-2, sampling='log')
#     model = models.Sequential([
#         layers.Input(shape=(X_train_scaled.shape[1],)),
#         layers.Dense(hp_units, activation='relu'),
#         layers.Dense(1, activation='sigmoid')
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr),
#         loss='binary_crossentropy',
#         metrics=[tf.keras.metrics.AUC(name='auc'), F1Metric()]
#     )
#     return model

# # --- Repeated tuning runs ----------------------------------------------------
# n_runs = 10
# auc_list = []
# f1_list = []

# for i in range(n_runs):
#     print(f"\n--- Run {i+1}/{n_runs} ---")

#     # Split and scale
#     X_train_df, X_test_df, y_train, y_test = train_test_split(
#         X, y, stratify=y, test_size=0.3, random_state=i)
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train_df)
#     X_test_scaled = scaler.transform(X_test_df)

#     # Class weights
#     classes = np.unique(y_train)
#     weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
#     class_weights = dict(zip(classes, weights))

#     # Clear old tuner directory
#     tuner_dir = f"keras_tuner_dir/run_{i}"
#     if os.path.exists(tuner_dir):
#         shutil.rmtree(tuner_dir)

#     tuner = kt.RandomSearch(
#         build_model,
#         objective='val_auc',
#         max_trials=2,
#         directory=tuner_dir,
#         project_name='one_hidden_layer4'
#     )

#     early_stop = callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=0)
#     tuner.search(
#         X_train_scaled, y_train,
#         epochs=3,
#         batch_size=512,
#         validation_split=0.2,
#         callbacks=[early_stop],
#         class_weight=class_weights,
#         verbose=0
#     )

#     best_model = tuner.get_best_models(1)[0]
#     y_proba_nn = best_model.predict(X_test_scaled).flatten()
#     auc = roc_auc_score(y_test, y_proba_nn)

#     best_t, best_f = 0, 0
#     for t in np.linspace(0.01, 0.99, 99):
#         f = f1_score(y_test, (y_proba_nn > t).astype(int))
#         if f > best_f:
#             best_f, best_t = f, t

#     print(f"AUC: {auc:.3f}, Best F1: {best_f:.3f} @ threshold {best_t:.2f}")
#     auc_list.append(auc)
#     f1_list.append(best_f)

# # --- Summary ----------------------------------------------------
# print("\n--- Final Summary (10 runs) ---")
# print(f"Mean AUC: {np.mean(auc_list):.3f} ± {np.std(auc_list):.3f}")
# print(f"Mean F1 : {np.mean(f1_list):.3f} ± {np.std(f1_list):.3f}")



# import os, shutil, numpy as np, tensorflow as tf, keras_tuner as kt
# from tensorflow.keras import layers, models, callbacks
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.preprocessing import StandardScaler
# from sklearn.utils.class_weight import compute_class_weight
# from sklearn.metrics import roc_auc_score, f1_score
# from sklearn.linear_model import LogisticRegression

# # ------------------------------------------------------------------
# #  DATA  (assume matched_clean DataFrame and predictors list exist)
# # ------------------------------------------------------------------
# X = matched_clean[predictors]
# y = matched_clean["event_type"]

# # ------------------------------------------------------------------
# #  CUSTOM F1 METRIC FOR KERAS
# # ------------------------------------------------------------------
# class F1Metric(tf.keras.metrics.Metric):
#     def __init__(self, name="f1", threshold=0.5, **kwargs):
#         super().__init__(name=name, **kwargs)
#         self.threshold = threshold
#         self.tp = self.add_weight(name="tp", initializer="zeros")
#         self.fp = self.add_weight(name="fp", initializer="zeros")
#         self.fn = self.add_weight(name="fn", initializer="zeros")

#     def update_state(self, y_true, y_pred, sample_weight=None):
#         y_pred = tf.cast(y_pred > self.threshold, tf.float32)
#         y_true = tf.cast(y_true, tf.float32)
#         self.tp.assign_add(tf.reduce_sum(y_true * y_pred))
#         self.fp.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
#         self.fn.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

#     def result(self):
#         precision = self.tp / (self.tp + self.fp + 1e-8)
#         recall    = self.tp / (self.tp + self.fn + 1e-8)
#         return 2 * precision * recall / (precision + recall + 1e-8)

#     def reset_states(self):
#         for v in self.variables:
#             v.assign(0)

# # ------------------------------------------------------------------
# #  KERAS‑TUNER MODEL BUILDER
# # ------------------------------------------------------------------
# def build_model(hp):
#     hp_units = hp.Int("units", 16, 256, step=16)
#     hp_lr    = hp.Float("lr", 1e-4, 1e-2, sampling="log")
#     model = models.Sequential(
#         [
#             layers.Input(shape=(X_train_scaled.shape[1],)),
#             layers.Dense(hp_units, activation="relu"),
#             layers.Dense(1, activation="sigmoid"),
#         ]
#     )
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr),
#         loss="binary_crossentropy",
#         metrics=[tf.keras.metrics.AUC(name="auc"), F1Metric()],
#     )
#     return model

# # ------------------------------------------------------------------
# #  CROSS‑VALIDATION LOOP (LR + NN on identical folds)
# # ------------------------------------------------------------------
# sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
# lr_auc, lr_f1, nn_auc, nn_f1 = [], [], [], []

# for fold, (train_idx, test_idx) in enumerate(sss.split(X, y)):
#     train_df, test_df = matched_clean.iloc[train_idx], matched_clean.iloc[test_idx]
#     X_train, y_train  = train_df[predictors], train_df["event_type"]
#     X_test,  y_test   = test_df[predictors],  test_df["event_type"]

#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled  = scaler.transform(X_test)

#     # ---------------- LOGISTIC REGRESSION ----------------
#     lr = LogisticRegression(max_iter=1000, class_weight="balanced")
#     lr.fit(X_train_scaled, y_train)
#     lr_proba = lr.predict_proba(X_test_scaled)[:, 1]
#     lr_auc.append(roc_auc_score(y_test, lr_proba))
#     lr_f1.append(max(f1_score(y_test, (lr_proba > t).astype(int))
#                      for t in np.linspace(0.01, 0.99, 99)))

#     # ---------------- NEURAL NET (KERAS‑TUNER) ----------------
#     classes  = np.unique(y_train)
#     cw       = dict(zip(classes, compute_class_weight("balanced", classes, y_train)))
#     tuner_dir = f"kt_run_{fold}"
#     if os.path.exists(tuner_dir): shutil.rmtree(tuner_dir)

#     tuner = kt.RandomSearch(
#         build_model,
#         objective="val_auc",
#         max_trials=5,
#         overwrite=True,
#         directory=tuner_dir,
#         project_name="nn",
#     )

#     es = callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=0)
#     tuner.search(
#         X_train_scaled,
#         y_train,
#         epochs=5,
#         batch_size=512,
#         validation_split=0.2,
#         class_weight=cw,
#         callbacks=[es],
#         verbose=0,
#     )

#     best_hp   = tuner.get_best_hyperparameters(1)[0]
#     best_model = build_model(best_hp)
#     best_model.fit(
#         X_train_scaled,
#         y_train,
#         epochs=5,
#         batch_size=512,
#         validation_split=0.2,
#         class_weight=cw,
#         callbacks=[es],
#         verbose=0,
#     )

#     nn_proba = best_model.predict(X_test_scaled).flatten()
#     nn_auc.append(roc_auc_score(y_test, nn_proba))
#     nn_f1.append(max(f1_score(y_test, (nn_proba > t).astype(int))
#                      for t in np.linspace(0.01, 0.99, 99)))

# # ------------------------------------------------------------------
# #  RESULTS
# # ------------------------------------------------------------------
# print(f"LR  AUC: {np.mean(lr_auc):.3f} ± {np.std(lr_auc):.3f}")
# print(f"LR  F1 : {np.mean(lr_f1):.3f} ± {np.std(lr_f1):.3f}")
# print(f"NN  AUC: {np.mean(nn_auc):.3f} ± {np.std(nn_auc):.3f}")
# print(f"NN  F1 : {np.mean(nn_f1):.3f} ± {np.std(nn_f1):.3f}")




Trial 20 Complete [00h 00m 23s]
val_auc: 0.8161504864692688

Best val_auc So Far: 0.825941264629364
Total elapsed time: 00h 08m 27s


C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 144)                 │           3,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             145 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,033 (15.75 KB)

 Trainable params: 4,033 (15.75 KB)

 Non-trainable params: 0 (0.00 B)

Best hyperparameters:
units: 144
lr: 0.005418161911653196
4173/4173 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step
AUC (NN): 0.824657841799013
Best threshold (NN): 0.8, F1: 0.17576923076923076
              precision    recall  f1-score   support

           0       0.98      0.95      0.97    130631
           1       0.12      0.31      0.18      2902

    accuracy                           0.94    133533
   macro avg       0.55      0.63      0.57    133533
weighted avg       0.97      0.94      0.95    133533



In [25]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import layers, models, callbacks
from sklearn.metrics import roc_auc_score, f1_score, classification_report
import numpy as np

from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))



# --- Custom F1 metric ---------------------------------------------------------
class F1Metric(tf.keras.metrics.Metric):
    def __init__(self, name='f1', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fp = self.add_weight(name='fp', initializer='zeros')
        self.fn = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        y_true = tf.cast(y_true, tf.float32)
        self.tp.assign_add(tf.reduce_sum(y_true * y_pred))
        self.fp.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
        self.fn.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

    def result(self):
        precision = self.tp / (self.tp + self.fp + 1e-8)
        recall   = self.tp / (self.tp + self.fn + 1e-8)
        return 2 * precision * recall / (precision + recall + 1e-8)

    def reset_states(self):
        for v in self.variables:
            v.assign(0)

# --- Model builder -----------------------------------------------------------
# --- Model builder -----------------------------------------------------------
def build_model(hp):
    hp_units_1 = hp.Int('units_1', min_value=16, max_value=256, step=16)
    hp_units_2 = hp.Int('units_2', min_value=16, max_value=256, step=16)
    hp_lr      = hp.Float('lr', 1e-4, 1e-2, sampling='log')
    model = models.Sequential([
        layers.Input(shape=(X_train_scaled.shape[1],)),
        layers.Dense(hp_units_1, activation='relu'),
        layers.Dense(hp_units_2, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc'), F1Metric()]
    )
    return model

# --- Hyperparameter tuning ----------------------------------------------------
tuner = kt.RandomSearch(
    build_model,
    objective='val_auc',
    max_trials=20,
    directory='keras_tuner_dir',
    project_name='two_hidden_layers_test2'
)

early_stop = callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=0)

tuner.search(
    X_train_scaled, y_train,
    epochs=3,
    batch_size=512,
    validation_split=0.2,
    callbacks=[early_stop],
    class_weight=class_weights,    
    verbose=1
)

# --- Evaluation ---------------------------------------------------------------
best_model = tuner.get_best_models(1)[0]

# Print model architecture
best_model.summary()
best_hp = tuner.get_best_hyperparameters(1)[0]

# Print all tuned hyperparameters
print("Best hyperparameters:")
for param in best_hp.values:
    print(f"{param}: {best_hp.get(param)}")

    

y_proba_nn = best_model.predict(X_test_scaled).flatten()

print("AUC (NN):", roc_auc_score(y_test, y_proba_nn))

best_t_nn, best_f_nn = 0, 0
for t in np.linspace(0.01, 0.99, 99):
    f = f1_score(y_test, (y_proba_nn > t).astype(int))
    if f > best_f_nn:
        best_f_nn, best_t_nn = f, t

print(f"Best threshold (NN): {best_t_nn}, F1: {best_f_nn}")
print(classification_report(y_test, (y_proba_nn > best_t_nn).astype(int)))

# import tensorflow as tf
# import keras_tuner as kt
# from tensorflow.keras import layers, models, callbacks
# from sklearn.metrics import roc_auc_score, f1_score, classification_report
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.utils.class_weight import compute_class_weight
# import numpy as np
# import os
# import shutil

# # --- Custom F1 metric ---------------------------------------------------------
# class F1Metric(tf.keras.metrics.Metric):
#     def __init__(self, name='f1', threshold=0.5, **kwargs):
#         super().__init__(name=name, **kwargs)
#         self.threshold = threshold
#         self.tp = self.add_weight(name='tp', initializer='zeros')
#         self.fp = self.add_weight(name='fp', initializer='zeros')
#         self.fn = self.add_weight(name='fn', initializer='zeros')

#     def update_state(self, y_true, y_pred, sample_weight=None):
#         y_pred = tf.cast(y_pred > self.threshold, tf.float32)
#         y_true = tf.cast(y_true, tf.float32)
#         self.tp.assign_add(tf.reduce_sum(y_true * y_pred))
#         self.fp.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
#         self.fn.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

#     def result(self):
#         precision = self.tp / (self.tp + self.fp + 1e-8)
#         recall   = self.tp / (self.tp + self.fn + 1e-8)
#         return 2 * precision * recall / (precision + recall + 1e-8)

#     def reset_states(self):
#         for v in self.variables:
#             v.assign(0)

# # --- Model builder -----------------------------------------------------------
# def build_model(hp):
#     hp_units_1 = hp.Int('units_1', min_value=16, max_value=256, step=16)
#     hp_units_2 = hp.Int('units_2', min_value=16, max_value=256, step=16)
#     hp_lr = hp.Float('lr', 1e-4, 1e-2, sampling='log')
#     model = models.Sequential([
#         layers.Input(shape=(X_train_scaled.shape[1],)),
#         layers.Dense(hp_units_1, activation='relu'),
#         layers.Dense(hp_units_2, activation='relu'),
#         layers.Dense(1, activation='sigmoid')
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=hp_lr),
#         loss='binary_crossentropy',
#         metrics=[tf.keras.metrics.AUC(name='auc'), F1Metric()]
#     )
#     return model

# # --- Repeated tuning runs ----------------------------------------------------
# n_runs = 10
# auc_list = []
# f1_list = []

# for i in range(n_runs):
#     print(f"\n--- Run {i+1}/{n_runs} ---")

#     # Split and scale
#     X_train_df, X_test_df, y_train_split, y_test = train_test_split(
#         X, y, stratify=y, test_size=0.3, random_state=i)
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train_df)
#     X_test_scaled = scaler.transform(X_test_df)

#     # Class weights
#     classes = np.unique(y_train_split)
#     weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train_split)
#     class_weights = dict(zip(classes, weights))

#     # Clear tuner directory
#     tuner_dir = f"keras_tuner_dir/two_hidden_run_{i}"
#     if os.path.exists(tuner_dir):
#         shutil.rmtree(tuner_dir)

#     tuner = kt.RandomSearch(
#         build_model,
#         objective='val_auc',
#         max_trials=20,
#         directory=tuner_dir,
#         project_name='two_hidden_layers3'
#     )

#     tuner.search(
#         X_train_scaled, y_train_split,
#         epochs=3,
#         batch_size=512,
#         validation_split=0.2,
#         callbacks=[callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=0)],
#         class_weight=class_weights,
#         verbose=0
#     )

#     best_model = tuner.get_best_models(1)[0]
#     y_proba_nn = best_model.predict(X_test_scaled).flatten()
#     auc = roc_auc_score(y_test, y_proba_nn)

#     best_t, best_f = 0, 0
#     for t in np.linspace(0.01, 0.99, 99):
#         f = f1_score(y_test, (y_proba_nn > t).astype(int))
#         if f > best_f:
#             best_f, best_t = f, t

#     print(f"AUC: {auc:.3f}, Best F1: {best_f:.3f} @ threshold {best_t:.2f}")
#     auc_list.append(auc)
#     f1_list.append(best_f)

# # --- Summary ----------------------------------------------------
# print("\n--- Final Summary (10 runs, 2 hidden layers) ---")
# print(f"Mean AUC: {np.mean(auc_list):.3f} ± {np.std(auc_list):.3f}")
# print(f"Mean F1 : {np.mean(f1_list):.3f} ± {np.std(f1_list):.3f}")


Trial 20 Complete [00h 01m 11s]
val_auc: 0.8315985202789307

Best val_auc So Far: 0.8315985202789307
Total elapsed time: 00h 11m 13s


C:\Users\magst\anaconda3\envs\APPENV2\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │           6,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 224)                 │          57,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             225 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 64,705 (252.75 KB)

 Trainable params: 64,705 (252.75 KB)

 Non-trainable params: 0 (0.00 B)

Best hyperparameters:
units_1: 256
units_2: 224
lr: 0.001141937009820313
4173/4173 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step
AUC (NN): 0.8338030391750468
Best threshold (NN): 0.8200000000000001, F1: 0.19841066393232504
              precision    recall  f1-score   support

           0       0.98      0.97      0.98    130631
           1       0.16      0.27      0.20      2902

    accuracy                           0.95    133533
   macro avg       0.57      0.62      0.59    133533
weighted avg       0.97      0.95      0.96    133533

